In [37]:
from recsys.inference import get_recommendations
from recsys.config import settings
import pickle
import torch
import pathlib
import pandas as pd

In [38]:
feature_dir = settings.PROCESSED_DATA_DIR
with open(feature_dir / 'features_dataset.pickle', 'rb') as handle:
    dataset = pickle.load(handle)

In [39]:
with open(feature_dir / 'events_df.pickle', 'rb') as handle:
    events_df = pickle.load(handle)
    
with open(feature_dir / 'bets_df.pickle', 'rb') as handle:
    bets_df = pickle.load(handle)

In [40]:
MODEL_REGISTRY = pathlib.Path().cwd().parent / 'model_registry'
model_save_path = MODEL_REGISTRY / 'two_tower_10_001_80_250130.pth'
model = torch.load(model_save_path, weights_only=False)

In [56]:
def analyze_user_behavior(user_id, bets_df, events_df):
    """
    Analyze user's betting patterns
    """
    user_bets = bets_df[bets_df['player_id'] == user_id]
    
    user_history = {
        'favorite_sports': user_bets.merge(events_df, on='event_id')['sport_id'].value_counts().to_dict(),
        'favorite_leagues': user_bets.merge(events_df, on='event_id')['league_id'].value_counts().to_dict(),
        'avg_odds': user_bets['bet_odds'].mean(),
        'preferred_market_types': user_bets['market_id'].value_counts().to_dict(),
        'successful_bets': user_bets[user_bets['status'] == 'won']['event_id'].tolist(),
        'betting_times': pd.to_datetime(user_bets['bet_date']).dt.hour.value_counts().to_dict()
    }
    
    return user_history

In [57]:
def generate_recommendations(model, dataset, user_id, top_k=10, device=settings.DEVICE):
    
    model.eval()  # Set model to evaluation mode
    
    with torch.no_grad():
        try:
            # Get user features
            user_idx = dataset.user_id_to_idx[user_id]
            user_features = dataset.user_feature_matrix[user_idx]
            user_features = torch.FloatTensor(user_features).unsqueeze(0).to(device)
            
            # Get all event features
            all_event_features = torch.FloatTensor(dataset.event_feature_matrix).to(device)
            
            # Expand user features to match event features dimension
            user_features_expanded = user_features.expand(len(dataset.event_features), -1)
            
            # Get scores for all events
            scores = model(user_features_expanded, all_event_features)
            
            # Get top-k recommendations
            top_scores, top_indices = torch.topk(scores, k=top_k)
            
            # Convert indices back to event IDs
            event_ids = list(dataset.event_id_to_idx.keys())
            recommended_events = [event_ids[idx] for idx in top_indices.cpu().numpy()]
            recommendation_scores = top_scores.cpu().numpy()
            
            # Create recommendations dataframe
            recommendations = pd.DataFrame({
                'event_id': recommended_events,
                'score': recommendation_scores
            })
            
            return recommendations
            
        except KeyError:
            print(f"User ID {user_id} not found in the dataset")
            return None


In [58]:

# Function to get event details
def get_event_details(recommendations, events_df):
    """
    Add event details to recommendations
    """
    return recommendations.merge(
        events_df[['event_id', 'sport_id', 'league_id', 'home_team', 'away_team', 'start_time']], 
        on='event_id',
        how='left'
    )


In [59]:

# Example usage:
def get_user_recommendations(model, dataset, user_id, events_df, top_k=10):
    """
    Get recommendations with full event details
    """
    # Get raw recommendations
    recommendations = generate_recommendations(model, dataset, user_id, top_k=top_k)
    
    if recommendations is not None:
        # Add event details
        detailed_recommendations = get_event_details(recommendations, events_df)
        
        # Sort by score
        detailed_recommendations = detailed_recommendations.sort_values('score', ascending=False)
        
        # Format score
        detailed_recommendations['score'] = detailed_recommendations['score'].round(4)
        
        return detailed_recommendations
    return None

In [60]:
def generate_explanation(recommendation, user_history, events_df):
    """
    Generate personalized explanation for a recommendation
    """
    explanations = []
    
    # Sport preference explanation
    if recommendation['sport_id'] in user_history['favorite_sports']:
        sport_rank = list(user_history['favorite_sports'].keys()).index(recommendation['sport_id']) + 1
        if sport_rank == 1:
            explanations.append("This is from your favorite sport")
        else:
            explanations.append(f"This is from one of your top {sport_rank} preferred sports")
    
    # League preference explanation
    if recommendation['league_id'] in user_history['favorite_leagues']:
        explanations.append("You've shown interest in this league before")
    
    # Team history explanation
    team_matches = events_df[
        (events_df['home_team'] == recommendation['home_team']) | 
        (events_df['away_team'] == recommendation['away_team'])
    ]
    if len(team_matches) > 0:
        if recommendation['event_id'] in user_history['successful_bets']:
            explanations.append("You've had successful bets on these teams before")
    
    # Time-based explanation
    event_hour = pd.to_datetime(recommendation['start_time']).hour
    if event_hour in user_history['betting_times']:
        explanations.append("This event is scheduled during your preferred betting times")
    
    # Similar event explanation
    if recommendation['score'] > 0.8:
        explanations.append("This closely matches your betting preferences")
    elif recommendation['score'] > 0.6:
        explanations.append("This aligns with your betting patterns")
    
    return " • ".join(explanations)

In [61]:
def generate_recommendations_with_explanations(model, dataset, user_id, events_df, user_history, top_k=10):
    """
    Generate recommendations with personalized explanations
    """
    # Get base recommendations
    recs = get_recommendations.generate_recommendations(model, dataset, user_id, top_k=top_k)
    
    if recs is None:
        return None
    
    # Add event details
    detailed_recs = get_recommendations.get_event_details(recs, events_df)
    
    # Generate explanations for each recommendation
    explanations = []
    for _, row in detailed_recs.iterrows():
        explanation = generate_explanation(row, user_history, events_df)
        explanations.append(explanation)
    
    detailed_recs['explanation'] = explanations
    
    return detailed_recs


In [62]:
def explain_recommendations(model, dataset, user_id, events_df, bets_df, top_k=10):
    """
    Generate and explain recommendations for a user
    """
    # Get user's betting history
    user_history = analyze_user_behavior(user_id, bets_df, events_df)
    
    # Generate recommendations with explanations
    recommendations = generate_recommendations_with_explanations(
        model, dataset, user_id, events_df, user_history, top_k
    )
    
    return recommendations

In [63]:
def print_recommendations_with_explanations(user_id, model, dataset, events_df, bets_df):
    """
    Print formatted recommendations with explanations
    """
    recommendations = explain_recommendations(model, dataset, user_id, events_df, bets_df)
    
    if recommendations is not None:
        print(f"\nPersonalized recommendations for user {user_id}:")
        print("-" * 100)
        
        for _, rec in recommendations.iterrows():
            print(f"Event: {rec['home_team']} vs {rec['away_team']}")
            print(f"League: {rec['league_id']}")
            print(f"Start Time: {rec['start_time']}")
            print(f"Confidence Score: {rec['score']:.4f}")
            print(f"Why this recommendation:")
            print(f"{rec['explanation']}")
            print("-" * 100)
    else:
        print(f"No recommendations found for user {user_id}")


In [64]:

# Advanced usage with filtering:
def get_filtered_recommendations(user_id, model, dataset, events_df, bets_df, 
                               sport_id=None, league_id=None, time_window=None):
    """
    Get filtered recommendations based on specific criteria
    """
    recommendations = explain_recommendations(model, dataset, user_id, events_df, bets_df)
    
    if recommendations is not None:
        # Apply filters
        if sport_id:
            recommendations = recommendations[recommendations['sport_id'] == sport_id]
        
        if league_id:
            recommendations = recommendations[recommendations['league_id'] == league_id]
        
        if time_window:
            start_time = pd.Timestamp.now()
            end_time = start_time + pd.Timedelta(hours=time_window)
            recommendations = recommendations[
                (pd.to_datetime(recommendations['start_time']) >= start_time) &
                (pd.to_datetime(recommendations['start_time']) <= end_time)
            ]
    
    return recommendations

In [65]:
user_id = 123  # Replace with actual user ID
print_recommendations_with_explanations(user_id, model, dataset, events_df, bets_df)



Personalized recommendations for user 123:
----------------------------------------------------------------------------------------------------
Event: World Cup 2026. Outrights vs nan
League: 1479
Start Time: 2026-08-20 10:00:00.000
Confidence Score: 3525819.5000
Why this recommendation:
This is from your favorite sport • This closely matches your betting preferences
----------------------------------------------------------------------------------------------------
Event: Spain. Liga Asobal. Outrights vs nan
League: 116
Start Time: 2025-06-05 20:00:00.000
Confidence Score: 788874.6875
Why this recommendation:
This closely matches your betting preferences
----------------------------------------------------------------------------------------------------
Event: Women. Wimbledon. Grass. Outrights vs nan
League: 1009
Start Time: 2025-07-13 13:00:00.000
Confidence Score: 474794.3125
Why this recommendation:
This closely matches your betting preferences
-----------------------------------

In [66]:
# Filtered recommendations
filtered_recs = get_filtered_recommendations(
    user_id=user_id,
    model=model,
    dataset=dataset,
    events_df=events_df,
    bets_df=bets_df,
)

filtered_recs

,event_id,score,sport_id,league_id,home_team,away_team,start_time,explanation
0,57602,3.525820e+06,3,1479,World Cup 2026. Outrights,NaN,2026-08-20 10:00:00.000,This is from your favorite sport • This closel...
1,61708,7.888747e+05,11,116,Spain. Liga Asobal. Outrights,NaN,2025-06-05 20:00:00.000,This closely matches your betting preferences
2,56366,4.747943e+05,5,1009,Women. Wimbledon. Grass. Outrights,NaN,2025-07-13 13:00:00.000,This closely matches your betting preferences
3,7217,4.406689e+05,3,100,Poland. Ekstraklasa. Outrights,NaN,2025-05-24 16:00:00.000,This is from your favorite sport • This closel...
4,30501,3.197098e+05,52,1070,The Open Championship. Outrights,NaN,2025-07-20 08:00:00.000,This closely matches your betting preferences
5,5610,2.502317e+05,53,1119,Tour de France. Outrights,NaN,2025-07-27 09:00:00.000,This closely matches your betting preferences
6,41848,1.506925e+05,3,106,Greece. Super League. Outrights,NaN,2025-05-01 16:00:00.000,This is from your favorite sport • This closel...
7,80486,1.414497e+05,18,1357,England. Premier League. Outrights,NaN,2025-06-01 05:00:00.000,This closely matches your betting preferences
8,77270,1.386047e+05,3,108,Romanian Cup. Outrights,NaN,2025-05-01 14:00:00.000,This is from your favorite sport • This closel...
9,52876,1.353638e+05,52,1070,US Open. Outrights,NaN,2025-06-15 10:00:00.000,This closely matches your betting preferences
